# About: Amazon S3へのバックアップ

---

Moodle構築環境のデータ、設定ファイルなどを Amazon S3 にバックアップします。

## 概要

アプリケーションテンプレートで構築したMoodle環境のバックアップを作成し Amazon S3 に保存します。

### 前提条件


この Notebook を実行するには事前に以下のものを準備する必要があります。

* バックアップ先となるAmazon S3のバケット
* Amazon S3のバケットに書き込みできるクレデンシャル

### バックアップの対象

バックアップの対象を以下に示します。

* DBデータ
* アップロードファイル(moodledata)
* PHPファイル
* 各コンテナで実行しているサービスの設定ファイル
    - Moodleコンテナ (`httpd.conf`, `php.ini`, ...)
    - データベースコンテナ (`my.cnf`, ...)
* `compose.yaml`
* コンテナイメージ
* crontab
* コンテナログのlogrotate設定
* /etc/rsyslogの設定ファイル(`051-ロギングドライバを変更する.ipynb`で設定した場合のみ)
* /etc/fluentdの設定ファイル(`052-ロギングドライバを変更する.ipynb`で設定した場合のみ)
* group_vars（`10-node.yml`は除く）

## パラメータ設定

### Ansibleのグループを指定する

バックアップ対象となるAnsibleのグループ名を指定してください。

既存のグループ名を確認するために`group_vars`にあるディレクトリの一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にしてAnsibleのグループ名を次のセルに指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group = 

#### チェック

指定された `target_group` の値が適切なものかチェックします。

`target_group` に対応する設定ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path

target_dir = Path('group_vars') / target_group
if not (target_dir.is_dir() and len(list(target_dir.glob("*"))) > 0):
    target_file = Path('group_vars') / f"{target_group}.yml"
    if target_file.is_file():
        target_dir.mkdir(exist_ok=True)
        target_file.rename(target_dir / "00-moodle.yml")
    else:
        raise RuntimeError(f"ERROR: not exists {target_group}")

バックアップ対象のノードにアクセスできることを確認します。

In [ ]:
!ansible {target_group} -m ping

### AWSの認証情報

Amazon S3を利用するための認証情報を指定します。

Amazonのアクセスキーを指定してください。

In [ ]:
from getpass import getpass
aws_access_key = getpass()

Amazonのシークレットキーを指定してください。

In [ ]:
aws_secret_key = getpass()

### バックアップの保存先を指定する

バックアップの保存先となる Amazon S3 のバケット名、パスを指定します。

Amazon S3 のバケット名を指定してください。

In [ ]:
# (例)
# s3_bucket = 's3-bucket-name'

s3_bucket = 

バックアップの保存先となるディレクトリを指定してください。バックアップファイルは、次のセルで指定したディレクトリにバックアップ日時に対応するサブディレクトリを作成し、その下に保存されます。

In [ ]:
# (例)
# backup_path = f'moodle-simple/{target_group}'

backup_path = 

### チェック

ここまでに入力したパラメータでAmazon S3を利用できることをチェックします。

指定されたバケットのファイル一覧を表示してみます。

In [ ]:
!ansible {target_group} -a \
    'env AWS_ACCESS_KEY_ID="{aws_access_key}" AWS_SECRET_ACCESS_KEY="{aws_secret_key}" \
    aws s3 ls s3://{s3_bucket}/'

## バックアップ

### メンテナンスモードへの切り替え

バックアップを作成する前にMoodleをメンテナンスモードに切り替えます。

メンテナンスモードのへの切り替えが必要ない場合は、次のセルの実行をスキップしてください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --enable'

### DBデータのバックアップ

DBデータのバックアップを作成します。

Amazon S3のバケット名、保存先のディレクトリと現在時刻からバックアップの保存先となるS3uriを生成します。

In [ ]:
import datetime
s3_prefix = f's3://{s3_bucket}/{backup_path}/{datetime.datetime.now().isoformat()}/'

s3_db_dest = s3_prefix + 'db.sql.gz'
print(s3_db_dest)

DBデータのバックアップを作成しAmazon S3に保存します。

> DBコンテナは MySQL/MariaDB であることを前提としています。

In [ ]:
!ansible {target_group} -m shell -a "chdir=/srv/moodle \
    docker compose exec db bash -c -o pipefail \
    'mariadb-dump --single-transaction -u\$(cat \$MYSQL_USER_FILE) \
    -p\$(cat \$MYSQL_PASSWORD_FILE) \$(cat \$MYSQL_DATABASE_FILE)' \
    | gzip | env AWS_ACCESS_KEY_ID={aws_access_key} AWS_SECRET_ACCESS_KEY='{aws_secret_key}' \
    aws s3 cp - {s3_db_dest}"

### その他ファイルのバックアップ

DBデータ以外のファイルのバックアップを作成します。

この節で作成するバックアップの対象を以下に示します。

* アップロードファイル(moodledata)
* PHPファイル
* 各コンテナで実行しているサービスの設定ファイル
    - Moodleコンテナ (`httpd.conf`, `php.ini`, ...)
    - データベースコンテナ (`my.cnf`, ...)
* `compose.yaml`
* crontab
* コンテナログのlogrotate設定
* /etc/rsyslogの設定ファイル(`051-ロギングドライバを変更する.ipynb`で設定した場合のみ)
* /etc/fluentdの設定ファイル(`052-ロギングドライバを変更する.ipynb`で設定した場合のみ)
* group_vars（`10-node.yml`は除く）

> ホスト環境の `/etc/` にある設定ファイルをバックアップの対象としていません（上記に示したものを除く）。必要に応じて別途バックアップを行ってください。

まずcrontabの設定をバックアップするために、設定内容をファイルに保存します。

In [ ]:
!ansible {target_group} -m file -a "path=/srv/moodle/misc state=directory"
!ansible {target_group} -m shell -a "chdir=/srv/moodle \
    crontab -l > misc/crontab"

コンテナログのlogrotate設定をバックアップするために、設定ファイルを `/srv/moodle/misc/logrotate.d` にコピーします。

In [ ]:
!ansible {target_group} -m file -a \
    "path=/srv/moodle/misc/logrotate.d state=directory"
!ansible {target_group} -b -a \
    "cp -fp /etc/logrotate.d/httpd /etc/logrotate.d/mysql-server \
    /srv/moodle/misc/logrotate.d"

rsyslogの設定ファイルをバックアップするために`/etc/rsyslog.conf`, `/etc/rsyslog.d/`を`/srv/moodle/misc/rsyslog`にコピーします。ただし`051-ロギングドライバを変更する.ipynb`でrsyslogのロギング設定を行った場合のみ実行します。

In [ ]:
%run scripts/group.py
try:
    logging_driver = load_group_var(target_group, "logging_driver")
except KeyError:
    logging_driver = None
if logging_driver == "syslog":
    !ansible {target_group} -m file -a \
        "path=/srv/moodle/misc/rsyslog state=directory"
    !ansible {target_group} -b -m shell -a \
        "rm -rf /srv/moodle/misc/rsyslog/* \
        && cp -a /etc/rsyslog.conf /etc/rsyslog.d /srv/moodle/misc/rsyslog"

fluentdの設定ファイルをバックアップするために`/etc/fluentd/`を`/srv/moodle/misc/fluentd`にコピーします。ただし`052-ロギングドライバを変更する.ipynb`でfluentdのロギング設定を行った場合のみ実行します。

In [ ]:
if logging_driver == "fluentd":
    !ansible {target_group} -m file -a \
        "path=/srv/moodle/misc/fluentd state=directory"
    !ansible {target_group} -b -m shell -a \
        "rm -rf /srv/moodle/misc/fluentd/* \
        && cp -a /etc/fluentd/* /srv/moodle/misc/fluentd/" || true

`group_vars`に記録したパラメータをバックアップするために、構築環境の`/srv/moodle/misc/group_vars` にコピーします。

In [ ]:
!ansible {target_group} -b -m shell -a 'chdir=/srv/moodle/misc \
        test -d group_vars \
        && mv group_vars .group_vars.$(date +"%Y%m%d%H%M%S")' \
    || true
!ansible {target_group} -m copy \
    -a 'src=group_vars/{target_group} dest=/srv/moodle/misc/group_vars/'

Moodleキャッシュなどのバックアップ対象から外すディレクトリを指定します。前節のバックアップ対象であるDBデータを格納しているディレクトリ `db/data`、データベースをリストアするためのSQLファイルを格納しているディレクトリ `db/sql`も除外対象としています。

In [ ]:
exclude_dir_list = [
    './moodle/data/moodledata/cache',
    './moodle/data/moodledata/localcache',
    './moodle/data/moodledata/sessions',
    './moodle/data/moodledata/temp',
    './moodle/data/moodledata/trashdir',
    './db/data',
    './db/sql',
    f'./misc/group_vars/{target_group}/10-node.yml',
]

除外ディレクトリのリストをファイルに書き込みます。

In [ ]:
from tempfile import TemporaryDirectory
from pathlib import Path

tmpdir = !ansible {target_group} -a 'mktemp -d'
with TemporaryDirectory() as workdir:
    exclude_file = Path(workdir) / 'exclude'
    with exclude_file.open(mode='w') as f:
        for line in exclude_dir_list:
            print(line, file=f)
    !cat {exclude_file}
    !ansible {target_group} -m copy -a 'src={str(exclude_file)} dest={tmpdir[1]}/'

バックアップの保存先となるS3uriを確認します。保存先のディレクトリはDBデータのバックアップの保存先と同じディレクトリとします。

In [ ]:
s3_moodle_dest = s3_prefix + 'moodle.tar.gz'
print(s3_moodle_dest)

バックアップを作成します。

In [ ]:
!ansible {target_group} -m shell -a \
    'bash -c "sudo tar czpf - -C /srv/moodle -X {tmpdir[1]}/exclude ." \
    | env AWS_ACCESS_KEY_ID={aws_access_key} AWS_SECRET_ACCESS_KEY="{aws_secret_key}" \
    aws s3 cp - {s3_moodle_dest}'

除外ディレクトリのファイルを格納していた作業ディレクトリを削除します。

In [ ]:
!ansible {target_group} -a 'rm -r {tmpdir[1]}'

### コンテナイメージ

コンテナイメージのバックアップを作成します。

> レポジトリから取得したコンテナイメージを変更せずに利用している場合は、この節をスキップすることも可能です。

Moodle環境を構成しているコンテナイメージ名を取得します。

In [ ]:
out = !ansible {target_group} -a 'chdir=/srv/moodle docker compose images'
images = [':'.join(x.split()[1:3]) for x in out[2:]]
for x in images:
    print(x)

バックアップの保存先となるS3uriを確認します。保存先のディレクトリは他のバックアップの保存先と同じディレクトリとします。

In [ ]:
s3_img_dest = s3_prefix + 'container-image.tar.gz'
print(s3_img_dest)

バックアップを作成します。

In [ ]:
!ansible {target_group} -m shell \
    -a 'docker save {" ".join(images)} \
    | gzip | env AWS_ACCESS_KEY_ID={aws_access_key} AWS_SECRET_ACCESS_KEY="{aws_secret_key}" \
    aws s3 cp - {s3_img_dest}'

### バックアップファイルの確認

作成したバックアップの一覧を表示し、バックアップが作成されたことを確認します。

In [ ]:
!ansible {target_group} -a \
    'env AWS_ACCESS_KEY_ID={aws_access_key} AWS_SECRET_ACCESS_KEY="{aws_secret_key}" \
    aws s3 ls {s3_prefix}'

### メンテナンスモードの解除

メンテナンスモードを解除し、バックアップ作業を終了します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --disable'